In [1]:
from dotenv import load_dotenv


load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI


llm = ChatOpenAI(model="qwen-max", temperature=0)

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate


def create_chain(prompt_template):
    """
    Create a LangChain chain with the given prompt template.

    Args:
        prompt_template (str): The prompt template string.

    Returns:
        LLMChain: A LangChain chain object.
    """
    prompt = PromptTemplate.from_template(prompt_template)
    return prompt | llm

# 直接任务规范

In [4]:
direct_task_prompt = """将以下文本的情绪分类为积极、消极或中性。不要解释你的理由，只需提供分类即可。

文本: {text}

情绪:"""

direct_task_chain = create_chain(direct_task_prompt)

# Test the direct task specification
texts = [
    "我非常喜欢这部电影！演员的表演非常精彩。",
    "对于一年中的这个时候来说，今天的天气相当典型。",
    "我对这家餐厅的服务感到失望。",
]

for text in texts:
    result = direct_task_chain.invoke({"text": text}).content
    print(f"文本: {text}")
    print(f"情续: {result}")

文本: 我非常喜欢这部电影！演员的表演非常精彩。
情续: 积极
文本: 对于一年中的这个时候来说，今天的天气相当典型。
情续: 中性
文本: 我对这家餐厅的服务感到失望。
情续: 消极


# 格式规范

In [5]:
format_spec_prompt = """生成一篇关于{topic}的简短新闻文章。
按照以下格式组织你的回复：

标题：[文章的吸引人的标题]

引言：[总结要点的简短介绍段落]

正文：[提供更多细节的 2-3 个简短段落]

结论：[结论句或行动号召]"""

format_spec_chain = create_chain(format_spec_prompt)

# Test the format specification prompting
topic = "发现一颗新的类地系外行星"
result = format_spec_chain.invoke({"topic": topic}).content
print(result)

标题：天文学家发现潜在宜居类地系外行星

引言：
在宇宙探索的最新进展中，国际天文团队宣布了一项令人兴奋的发现——一颗位于其恒星星系可居住带内的新类地行星。这颗被命名为Kepler-452b的行星不仅大小与地球相近，而且围绕着一颗类似太阳的恒星运转，为寻找外星生命提供了新的希望。

正文：
据研究小组介绍，通过NASA的开普勒太空望远镜观测数据进行分析后确认了这一发现。Kepler-452b距离我们约1,400光年，在天鹅座方向。它所处的位置允许液态水存在于表面的可能性极大，这是支持生命存在的关键条件之一。此外，该行星绕行其母星一周的时间约为385个地球日，进一步增加了它与地球之间的相似性。
科学家们还指出，尽管目前尚无法直接探测到Kepler-452b的大气层组成情况，但基于其质量估计（大约是地球质量的五倍），推测其可能拥有岩石质地，并且存在维持大气层的能力，这对于未来深入研究该星球是否适合生命居住至关重要。

结论：
随着技术的进步和更多先进设备投入使用，人类对遥远世界的了解将越来越深刻。此次发现不仅拓展了我们对于宇宙的认识边界，也为解答“我们在宇宙中是否孤独”这个古老问题带来了新的线索。接下来，研究人员计划利用更强大的望远镜继续观察Kepler-452b及其周围环境，希望能够揭开更多关于这颗神秘行星的秘密。


# 多步骤推理

In [6]:
multi_step_prompt = """分析以下文本的主要论点、支持证据和潜在反驳。
按以下步骤提供分析：

1. 主要论点：确定并陈述主要主张或论点。

2. 支持证据：列出用于支持主要论点的关键点或证据。

3. 潜在反驳：提出对主要论点的可能反对意见或其他观点。

文本：{text}

分析："""

multi_step_chain = create_chain(multi_step_prompt)

# Test the multi-step reasoning approach
text = """虽然电动汽车经常被吹捧为解决气候变化的方案，但它们对环境的影响并不像看上去那么简单。
生产电动汽车电池需要大量的采矿作业，这可能导致栖息地破坏和水污染。
此外，如果用于为这些车辆充电的电力来自化石燃料，总体碳足迹可能不会显著减少。
然而，随着可再生能源变得越来越普遍和电池技术的进步，电动汽车确实可以在应对气候变化方面发挥关键作用。"""

result = multi_step_chain.invoke({"text": text}).content
print(result)

1. 主要论点：
本文的主要论点是电动汽车虽然被视为应对气候变化的有效工具，但它们当前对环境的影响并不完全积极，存在一些负面影响。不过，随着技术进步和能源结构的转变，未来电动汽车在减缓气候变化方面仍可扮演重要角色。

2. 支持证据：
- 电动汽车电池生产过程中需要大量矿物资源，这将导致栖息地破坏及水资源污染。
- 当前许多地方用于给电动汽车供电的电能主要来源于化石燃料发电厂，这意味着从整个生命周期来看，电动汽车并没有显著降低温室气体排放量。
- 随着可再生能源比例增加以及电池技术不断改进，预计将来电动汽车能够更有效地帮助减少全球碳足迹。

3. 潜在反驳：
- 对于第一个支持点（关于采矿活动造成的环境损害），有人可能会提出目前已经有企业在努力提高矿产开采效率并采取措施减轻其对自然环境的影响；同时，回收利用废旧电池也是解决这一问题的一种方式。
- 第二个支持点提到如果电力来源依旧是化石燃料，则电动车辆并不能真正实现低碳出行目标。对此，反对者可能会指出即便如此，在城市中心使用电动车仍然有助于改善当地空气质量，并且随着电网中清洁能源占比逐渐提升，长远来看还是有利于减排的。
- 最后一个观点认为随着时间推移和技术进步，电动汽车将成为对抗气候变暖的有效手段。然而也存在质疑声音认为技术革新速度可能无法赶上气候变化的步伐，或者认为即使有了更加环保高效的电动车，如果没有足够多的人愿意购买和使用它们，那么这种潜力就很难转化为现实效果。


# 比较分析

In [7]:
def compare_prompts(task, prompt_templates):
    """
    Compare different prompt templates for the same task.

    Args:
        task (str): The task description or input.
        prompt_templates (dict): A dictionary of prompt templates with their names as keys.
    """
    print(f"任务: {task}\n")
    for name, template in prompt_templates.items():
        chain = create_chain(template)
        result = chain.invoke({"task": task}).content
        print(f"{name} 提示结果:")
        print(result)
        print("\n" + "-" * 50 + "\n")


task = "简明阐述区块链技术的概念"

prompt_templates = {
    "基础": "解释{task}.",
    "结构化": """通过解决以下几点来解释{task}：
1. 定义
2. 主要特征
3. 实际应用
4. 对行业的潜在影响""",
}

compare_prompts(task, prompt_templates)

任务: 简明阐述区块链技术的概念

基础 提示结果:
区块链技术是一种分布式数据库技术，它通过维护一个持续增长的记录列表（称为“区块”）来存储数据。每个区块都包含一定数量的交易信息，并且通过加密方法与前一个区块链接起来，形成一条链。这种结构使得一旦信息被记录下来就很难被篡改或删除，因为任何对单个区块内容的修改都会影响到后续所有区块的数据一致性。

区块链的核心特点包括：

1. **去中心化**：传统上，数据存储和管理依赖于中央服务器或机构。而在区块链中，数据分散在网络中的多个节点上，没有单一控制点。
2. **透明性**：虽然参与者可以匿名操作，但所有交易都是公开可见的，增加了系统的透明度。
3. **不可篡改性**：由于每个新区块都会链接到前面的区块，并使用复杂的算法保证其安全性，因此一旦数据被写入区块链就几乎不可能被更改。
4. **安全性**：利用先进的加密技术和共识机制（如工作量证明PoW、权益证明PoS等），确保只有合法用户才能添加新的交易到区块链上。

区块链最初是作为比特币背后的技术而闻名，但现在已经被广泛应用于金融、供应链管理、医疗健康等多个领域，以提高效率、降低成本并增强信任。

--------------------------------------------------

结构化 提示结果:
### 区块链技术简明阐述

#### 1. 定义
区块链是一种分布式账本技术，它通过网络中多个节点共同维护一个持续增长的记录列表（称为“区块”），每个区块都包含了多条交易信息，并且每个新区块都会链接到前一个区块上形成链条结构。这一设计确保了数据一旦被记录就无法轻易修改或删除，从而提供了一种安全透明的方式来存储和传输价值。

#### 2. 主要特征
- **去中心化**：没有单一控制点；所有参与者都有权访问整个数据库。
- **透明性**：虽然个人身份可以保持匿名状态，但所有交易都是公开可见的。
- **不可篡改性**：一旦信息经过验证并添加到区块链中，就会永久保存下来，除非能够同时控制系统中超过50%以上的节点，否则很难更改历史数据。
- **安全性高**：利用复杂的加密算法保护数据不被盗用或伪造。
- **自治性**：基于预设规则自动执行智能合约等功能，减少人为干预的需求。

#### 3. 实际应用
- **金融服务**：如比特币等加密货币